In [ ]:
#load packages
library(ggplot2)
library(grid)
library(plyr)
library(gridExtra)
library(pspearman)
library(arivale.data.interface)

In [ ]:
#load dataframe
metab<-read.csv('statins_df.csv')
#check dimensions
metab<-metab[complete.cases(metab$chem_vendor), ]
dim(metab)
#subset LCA lab vendor
LCA<-metab[which(metab$chem_vendor=='LCA'),]
dim(LCA)
#subset Quest lab vendor
Quest<-metab[which(metab$chem_vendor=='Quest'),]
dim(Quest)
count(Quest,'statins_binary')
count(LCA,'statins_binary')

In [ ]:
#generate figure 1d for Quest diagnostics
grp.mean = ddply(Quest, .(statins_binary), summarise, mean=mean(HMG))
p <- ggplot(Quest, aes(x = LDL, y = HMG, color = factor(statins_binary))) 
p <- p + geom_point(alpha = 0.95,size=2) + geom_smooth(method = 'lm')+geom_point(shape = 1,colour = "black",alpha = 0.25)
p <- p + facet_wrap(~factor(statins_binary, labels = c(0,1)))+scale_fill_manual(values = c("lightskyblue",'red'))
p <- p + theme_bw()+theme(legend.position = 'none',
               axis.title = element_text(size = 10),
               axis.text = element_text(size = 21),
               strip.text.x = element_text(size = 18))
#p <- p + xlab('LDL Cholesterol') + ylab('HMG')
p <- p + scale_y_continuous(limits = c(.25,4.5), expand = c(.025,.025))
p <- p + scale_color_manual(values = c("lightskyblue",'darkred'))

dY <- ggplot(Quest, aes(HMG, fill = factor(statins_binary, labels = c(0,1))))
dY <- dY + geom_density(alpha = 0.95) + coord_flip()
dY <- dY + scale_x_continuous(limits = c(.25,4.75), expand = c(.025,.025))
dY<- dY+scale_fill_manual(values = c("lightskyblue",'darkred'))
dY <- dY + theme_bw()+theme(legend.position = "none",
                     panel.grid.major.x = element_blank(),
                     panel.grid.minor.x = element_blank(),
                     axis.ticks = element_blank(),
                     axis.title = element_blank(),
                     axis.text = element_blank(),
                     plot.margin = unit(c(10,5,12,-2), "mm"))
dY <- dY
dY <- dY + geom_vline(data = grp.mean, mapping = aes(xintercept = mean), lwd = 1.2)
options(repr.plot.width=10)

In [ ]:
#show figure 1d Quest Diagnostics
grid.arrange(p, dY, ncol=2, nrow=1, widths=c(4, .8), heights=c(5))

In [ ]:
#generate figure 1d for LCA
grp.mean = ddply(LCA, .(statins_binary), summarise, mean=mean(HMG))
p <- ggplot(LCA, aes(x = LDL, y = HMG, color = factor(statins_binary))) 
p <- p + geom_point(alpha = 0.95,size=2) + geom_smooth(method = 'lm')+geom_point(shape = 1,colour = "black",alpha = 0.25)
p <- p + facet_wrap(~factor(statins_binary, labels = c(0,1)))+scale_fill_manual(values = c("lightskyblue",'red'))
p <- p + theme_bw()+theme(legend.position = 'none',
               axis.title = element_text(size = 10),
               axis.text = element_text(size = 21),
               strip.text.x = element_text(size = 18))
#p <- p + xlab('LDL Cholesterol') + ylab('HMG')
p <- p + scale_y_continuous(limits = c(.25,4.75), expand = c(.025,.025))
p <- p + scale_color_manual(values = c("lightskyblue",'darkred'))

dY <- ggplot(LCA, aes(HMG, fill = factor(statins_binary, labels = c(0,1))))
dY <- dY + geom_density(alpha = 0.95) + coord_flip()
dY <- dY + scale_x_continuous(limits = c(0.25,4.75), expand = c(.025,.025))
dY<- dY+scale_fill_manual(values = c("lightskyblue",'darkred'))
dY <- dY + theme_bw()+theme(legend.position = "none",
                     panel.grid.major.x = element_blank(),
                     panel.grid.minor.x = element_blank(),
                     axis.ticks = element_blank(),
                     axis.title = element_blank(),
                     axis.text = element_blank(),
                     plot.margin = unit(c(10,5,12,-2), "mm"))
dY <- dY
dY <- dY + geom_vline(data = grp.mean, mapping = aes(xintercept = mean), lwd = 1.2)
options(repr.plot.width=10)

In [ ]:
#show figure 1d LCA
grid.arrange(p, dY, ncol=2, nrow=1, widths=c(4, .8), heights=c(5))

In [ ]:
#run statistics for Figure 1d (Quest)
no_statins<-Quest[which(!Quest$statins_binary %in% c(1)),]
statins<-Quest[which(Quest$statins_binary %in% c(1)),]

In [ ]:
#spearman correlation between HMG and LDL in statin users and non-users seperately
print('statins LDL-HMG spearman')
spearman.test(c(statins$HMG),c(statins$LDL))
print('non-statins LDL-HMG spearman')
spearman.test(c(no_statins$HMG),c(no_statins$LDL))

In [ ]:
library(MASS)
#run GLM adjusting for covariates modeling HMG-LDL relationship in statin users
#scale and center LDL
statins$LDL_scaled<-scale(statins$LDL)
#run GLM regression adjusting for covariates for figure 1d
summary(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=statins,family = Gamma(link = "log")))
confint(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=statins,family = Gamma(link = "log")))

In [ ]:
#run GLM adjusting for covariates modeling HMG-LDL relationship in non users
#scale and center LDL for non-statin group
no_statins$LDL_scaled<-scale(no_statins$LDL)
#run linear regression adjusting for covariates for figure 1d
summary(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=no_statins,family = Gamma(link = "log")))
confint(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=no_statins,family = Gamma(link = "log")))

In [ ]:
#run statistics for Figure 1d (LCA)
no_statins<-LCA[which(!LCA$statins_binary %in% c(1)),]
statins<-LCA[which(LCA$statins_binary %in% c(1)),]
print('statins LDL-HMG spearman (LCA)')
spearman.test(c(statins$HMG),c(statins$LDL))
print('non-statins LDL-HMG spearman (LCA)')
spearman.test(c(no_statins$HMG),c(no_statins$LDL))

In [ ]:
#scale and center LDL
statins$LDL_scaled<-scale(statins$LDL)
#run GLM regression adjusting for covariates for figure 1d
summary(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=statins,family = Gamma(link = "log")))
confint(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=statins,family = Gamma(link = "log")))

In [ ]:
#scale and center LDL
no_statins$LDL_scaled<-scale(no_statins$LDL)
#run GLM regression adjusting for covariates for figure 1d
summary(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=no_statins,family = Gamma(link = "log")))
confint(glm(HMG~BMI+age+age_sq+sex+LDL_scaled,data=no_statins,family = Gamma(link = "log")))

In [ ]:
#obtain p-value for difference in HMG between statin users and non-users within Quest and LCA labs, adjusting for covariates
print('Quest')
coef(summary(glm(HMG~BMI+age+age_sq+sex+statins_binary,data=Quest,family = Gamma(link = "log"))))[, "Pr(>|t|)"]
print("LCA")
coef(summary(glm(HMG~BMI+age+age_sq+sex+statins_binary,data=LCA,family = Gamma(link = "log"))))[, "Pr(>|t|)"]

In [ ]:
coef(summary(glm(HMG~BMI+age+age_sq+sex+statins_binary,data=Quest,family = Gamma(link = "log"))))
confint(glm(HMG~BMI+age+age_sq+sex+statins_binary,data=Quest,family = Gamma(link = "log")))

In [ ]:
(coef(summary(glm(HMG~BMI+age+age_sq+sex+statins_binary,data=LCA,family = Gamma(link = "log")))))
(confint(glm(HMG~BMI+age+age_sq+sex+statins_binary,data=LCA,family = Gamma(link = "log"))))